In [ ]:
# Import necessary libraries
import pygame
import numpy as np
import heapq

# Initialize Pygame
pygame.init()

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Set the dimensions of the grid
GRID_WIDTH = 20
GRID_HEIGHT = 20
CELL_SIZE = 20

# Create the grid
grid = np.zeros((GRID_HEIGHT, GRID_WIDTH))

# Define the Node class
class Node:
    def __init__(self, row, col):
        self.row = row
        self.col = col
        self.g = float('inf')  # Cost from start to node
        self.h = float('inf')  # Heuristic cost to end
        self.f = float('inf')  # Total cost
        self.parent = None
        self.wall = False

    def __lt__(self, other):
        return self.f < other.f

# Initialize the grid with nodes
nodes = [[Node(row, col) for col in range(GRID_WIDTH)] for row in range(GRID_HEIGHT)]

# Heuristic function (Manhattan distance)
def heuristic(a, b):
    return abs(a.row - b.row) + abs(a.col - b.col)

# A* Pathfinding algorithm
def astar(start, end):
    open_set = []
    heapq.heappush(open_set, start)
    start.g = 0
    start.h = heuristic(start, end)
    start.f = start.h

    while open_set:
        current = heapq.heappop(open_set)

        if current == end:
            return reconstruct_path(current)

        for neighbor in get_neighbors(current):
            tentative_g = current.g + 1  # Distance to the neighbor

            if tentative_g < neighbor.g:
                neighbor.parent = current
                neighbor.g = tentative_g
                neighbor.h = heuristic(neighbor, end)
                neighbor.f = neighbor.g + neighbor.h

                if neighbor not in open_set:
                    heapq.heappush(open_set, neighbor)

    return []  # No path found

# Dijkstra's Pathfinding algorithm
def dijkstra(start, end):
    open_set = []
    heapq.heappush(open_set, start)
    start.g = 0

    while open_set:
        current = heapq.heappop(open_set)

        if current == end:
            return reconstruct_path(current)

        for neighbor in get_neighbors(current):
            tentative_g = current.g + 1  # Distance to the neighbor

            if tentative_g < neighbor.g:
                neighbor.parent = current
                neighbor.g = tentative_g

                if neighbor not in open_set:
                    heapq.heappush(open_set, neighbor)

    return []  # No path found

# Function to get neighbors of a node
def get_neighbors(node):
    neighbors = []
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        x, y = node.row + dx, node.col + dy
        if 0 <= x < GRID_HEIGHT and 0 <= y < GRID_WIDTH:
            neighbor = nodes[x][y]
            if not neighbor.wall:  # Only consider walkable nodes
                neighbors.append(neighbor)
    return neighbors

# Function to reconstruct the path from start to end
def reconstruct_path(current):
    path = []
    while current:
        path.append(current)
        current = current.parent
    return path[::-1]  # Return reversed path

# Function to draw the grid
def draw_grid(screen):
    for row in range(GRID_HEIGHT):
        for col in range(GRID_WIDTH):
            color = WHITE
            node = nodes[row][col]
            if node.wall:
                color = BLACK
            pygame.draw.rect(screen, color, (col * CELL_SIZE, row * CELL_SIZE, CELL_SIZE, CELL_SIZE))

# Function to draw the path
def draw_path(screen, path):
    for node in path:
        pygame.draw.rect(screen, GREEN, (node.col * CELL_SIZE, node.row * CELL_SIZE, CELL_SIZE, CELL_SIZE))

# Main function
def main():
    screen = pygame.display.set_mode((GRID_WIDTH * CELL_SIZE, GRID_HEIGHT * CELL_SIZE))
    pygame.display.set_caption("Pathfinding Visualization")
    
    # Create some walls
    for i in range(5, 15):
        nodes[10][i].wall = True

    # Set start and end points
    start = nodes[0][0]
    end = nodes[GRID_HEIGHT - 1][GRID_WIDTH - 1]

    running = True
    algorithm = 'A*'  # Change to 'Dijkstra' for Dijkstra's algorithm

    while running:
        screen.fill(WHITE)
        draw_grid(screen)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_a:
                    path = astar(start, end)
                elif event.key == pygame.K_d:
                    path = dijkstra(start, end)

        if 'path' in locals():
            draw_path(screen, path)

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()